# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
gmaps.configure(api_key=g_key)
gmaps.figure()

Figure(layout=FigureLayout(height='420px'))

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
read_csv='cities.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
pd.read_csv('cities.csv')
cities= pd.read_csv('cities.csv')

In [5]:
location_df=cities[['Lat','Lng']]

In [6]:
humidity_df=cities['Humidity']

In [7]:
from api_keys import g_key
gmaps.configure(api_key=g_key)
gmaps.figure()

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
reduced_cities_df = cities.loc[(cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0) & \
                                   (cities["Max Temp"] > 70) & (cities["Max Temp"] < 80)].dropna()
reduced_cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = reduced_cities_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [10]:
import pprint
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    pprint.pprint(response)
    # extract results
    results = response['results']
 # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
    print("------------")

Retrieving Results for Index 70: ponta do sol.
{'html_attributions': [],
 'next_page_token': 'Aap_uEC_kCSZuAKgQ5H7bOIo3VRJlmNFR6fD3LTSnFo0MMmxquFxYzq-XfIoubwFhCD3msSxBD8zp0yTHhSiyuEP45t-TkKyE42qQuyzauwziEPTl8fVJ-Zy1rvfIii5B1bt9cDHTlKCJzIpV3MPjpad5cb8aFxqj1rGfFMcfIwPnmDli8hnVroc7h8VUUqU1QASJkzc3IedVk41HNEazSs0q087KMIjN_9pSASYzM6_UE1wD7LkeozasKOyTA8E23S3xW-_j3nOF_3EW0vkZKQSlehdR4d7wpBytDrjujCddgKTgkxqekuF-Lpy4qYxsI_PJNwDKKJf-6Y45IWZHPZm_mS8bWlBU4MTTJMFLbn9HC30F8TLlFaqWPsBe6Re4SdYpmPEvjA912XuKoi0EssNL6rd4VoH3Muw2iIAakjDHZQHxgQFwhLersUz9rBTOBY',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.6289793, 'lng': -46.0285609},
                           'viewport': {'northeast': {'lat': -20.62777447010728,
                                                      'lng': -46.02738632010728},
                                        'southwest': {'lat': -20.63047412989272,
                                                      'lng': -46.03008597989272

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -30.1778807, 'lng': -50.2035006},
                           'viewport': {'northeast': {'lat': -30.17650502010729,
                                                      'lng': -50.20220167010728},
                                        'southwest': {'lat': -30.17920467989273,
                                                      'lng': -50.20490132989272}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Hotel Castelo',
              'photos': [{'height': 3456,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/103409768676331094316">Delci '
    

{'html_attributions': [],
 'next_page_token': 'Aap_uEALB9OWPY2_kyyKLjlfTfkzIt0lTtPFhrclTFZISrvjxK2XMHz_Em-7lJAu70pU0C2rDq8_9tzole0Uwm2X8UcUKuc1xj9TqQRC0WVlrl01qDk5G-Ki3u24tF4qtgZlyLVVwAy-npCMqPFNQvBRHLuFRcTiCO_SpsialcYvvPModT6EcCJOjK2T2vbUt8LPmW4czy5bKhLkC1CPMdz2iVoUTTjGvyrZlRCSbauN2WKY2c7U3rkFDIP44o0pB28aqXNKwTrndCqTVrMiRQgd3UeN14zIxfiAbUxmkkJPjX7muomAH6GPI1-I3qTjRc6pyaTWaLpQx-Of8_nn0EW48np_Zu_wETjlEYsC2jq5Xjpg3vTJCwiKZvp7-Bwv6RyZawYFvPJSMFKomtr1j3wUmQ80MxlUhyPL7v7Dy_w6QivBEjFkNxDLJPeBwA7Al50',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.3000259, 'lng': 44.2679367},
                           'viewport': {'northeast': {'lat': -20.29871172010727,
                                                      'lng': 44.26928077989272},
                                        'southwest': {'lat': -20.30141137989272,
                                                      'lng': 44.26658112010728}}},
              'icon': 'https://maps.gstatic.c

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 15.3476121, 'lng': 74.482091},
                           'viewport': {'northeast': {'lat': 15.34857382989272,
                                                      'lng': 74.48751889999998},
                                        'southwest': {'lat': 15.34587417010728,
                                                      'lng': 74.4802817}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Dandeli Dreams Jungle Resort',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 2448,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 22.9096939, 'lng': 96.5038617},
                           'viewport': {'northeast': {'lat': 22.91113002989272,
                                                      'lng': 96.50508842989272},
                                        'southwest': {'lat': 22.90843037010728,
                                                      'lng': 96.50238877010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Mount Mogok Hotel',
              'photos': [{'height': 607,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/111175695056602902702">A '
           

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 32.8799738,
                                        'lng': 59.21613479999999},
                           'viewport': {'northeast': {'lat': 32.88132187989272,
                                                      'lng': 59.21742852989271},
                                        'southwest': {'lat': 32.87862222010727,
                                                      'lng': 59.21472887010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'مهمان پذیر نوید',
              'place_id': 'ChIJLTVncPZnGj8Re9U-HKuHLWY',
              'rating': 3.5,
              'reference': 'ChIJLTVncPZnGj8Re9U-HKuHLWY',
              'scope': 'GOOGLE',
   

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
fig = gmaps.figure()

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))